# 프로젝트: Spectrogram classification 모델 구현

- 앞에서는 **`1차원 Waveform` 데이터를 입력**받아 **Text 라벨을 출력하는 모델**을 기본 버전과 Skip-connection 버전으로 나누어 학습시켜 봄. (**`ConV1D` 이용**)

- 이번 프로젝트에서는 **`2차원 Spectrogram` 데이터를 입력**받아 **Text 라벨을 출력하는 모델**을 아래 제시된 단계와 같이 진행. (**`Wav2vec` 이용**)

- 기본 버전과 Skip-connection 버전으로 나누어 각각 진행


---

1. 데이터 처리와 분류

   - 라벨 데이터 처리하기
   - sklearn의 train_test_split함수를 이용하여 train, test 분리
   

2. 학습을 위한 하이퍼파라미터 설정


3. 데이터셋 구성

   - tf.data.Dataset을 이용
   - from_tensor_slices 함수에 return 받길 원하는 데이터를 튜플 (data, label) 형태로 넣어서 사용
   - map과 batch를 이용한 데이터 전처리
   - 주의 : waveform을 spectrogram으로 변환하기 위해 추가로 사용하는 메모리 때문에 이후 메모리 부족 현상을 겪게 될수도 있습니다.
   - tf.data.Dataset이 생성된 이후, 아래 예시와 같이 wav 데이터나 spectrogram 데이터를 담아둔 메모리 버퍼를 비워 주면 도움이 됩니다.
   

> del speech_data
> del spec_data


4. 2차원 Spectrogram 데이터를 처리하는 모델 구성

   - 2차원 Spectrogram 데이터의 시간축 방향으로 Conv1D layer를 적용, 혹은 Conv2D layer를 적용 가능
   - batchnorm, dropout, dense layer 등을 이용
   - 12개의 단어 class를 구분하는 loss를 사용하고 Adam optimizer를 사용
   - 모델 가중치를 저장하는 checkpoint callback 함수 추가
   - 다양한 모델의 실험을 진행해 보시기 바랍니다.
   
   

5. 학습 후, 학습이 어떻게 진행됐는지 그래프로 출력

   - loss, accuracy를 그래프로 표현



6. Test dataset을 이용해서 모델의 성능을 평가

   - 저장한 weight 불러오기
   - 모델의 예측값과 정답값이 얼마나 일치하는지 확인
    
---


In [5]:
import numpy as np
import matplotlib.pyplot as plt
import os
import IPython.display as ipd
import random
import librosa

data_path = os.getenv("HOME")+'/Exploration_mj/speech_recognition/data/speech_wav_8000.npz'
speech_data = np.load(data_path)

## 1. 데이터 처리와 분류

   - 라벨 데이터 처리하기
   - sklearn의 train_test_split함수를 이용하여 train, test 분리   

In [7]:
def wav2spec(wav, fft_size=258): # spectrogram shape을 맞추기위해서 size 변형
    D = np.abs(librosa.stft(wav, n_fft=fft_size))
    return D

# 위에서 뽑았던 sample data
spec = wav2spec(speech_data)
print("Waveform shape : ",data.shape)
print("Spectrogram shape : ",spec.shape)

ParameterError: Audio data must be of type numpy.ndarray

In [4]:
target_list = ['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go']

label_value = target_list
label_value.append('unknown')
label_value.append('silence')

print('LABEL : ', label_value, '\n')

# 학습에 사용하기 위해   Text 라벨 데이터를 => index로 바꿔주는 작업
new_label_value = dict()
for i, l in enumerate(label_value):
    new_label_value[l] = i
label_value = new_label_value

print('Indexed LABEL : ', new_label_value, '\n')

# index 작업을 통해서 Label data를 더 쉽게 사용
temp = []
for v in speech_data["label_vals"]:
    temp.append(label_value[v[0]])
label_data = np.array(temp)

print('label_data : ',label_data)

LABEL :  ['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go', 'unknown', 'silence'] 

Indexed LABEL :  {'yes': 0, 'no': 1, 'up': 2, 'down': 3, 'left': 4, 'right': 5, 'on': 6, 'off': 7, 'stop': 8, 'go': 9, 'unknown': 10, 'silence': 11} 

label_data :  [ 3  3  3 ... 11 11 11]


In [ ]:
from sklearn.model_selection import train_test_split

sr = 8000
train_wav, test_wav, train_label, test_label = train_test_split(speech_data["wav_vals"], 
                                                                label_data, 
                                                                test_size=0.3,
                                                                shuffle=True)
print(train_wav)

train_wav = train_wav.reshape([-1, sr, 1]) # add channel for CNN
test_wav = test_wav.reshape([-1, sr, 1])

## 2. 학습을 위한 하이퍼파라미터 설정

In [ ]:
batch_size = 32
max_epochs = 10

# the save point
checkpoint_dir = os.getenv('HOME')+'/Exploration_mj/speech_recognition/models/wav'

checkpoint_dir

## 3. 데이터셋 구성

   - tf.data.Dataset을 이용
   - from_tensor_slices 함수에 return 받길 원하는 데이터를 튜플 (data, label) 형태로 넣어서 사용
   - map과 batch를 이용한 데이터 전처리
   - 주의 : waveform을 spectrogram으로 변환하기 위해 추가로 사용하는 메모리 때문에 이후 메모리 부족 현상을 겪게 될수도 있습니다.
   - tf.data.Dataset이 생성된 이후, 아래 예시와 같이 wav 데이터나 spectrogram 데이터를 담아둔 메모리 버퍼를 비워 주면 도움이 됩니다.
   

> del speech_data
> del spec_data

In [ ]:
def one_hot_label(wav, label):
    label = tf.one_hot(label, depth=12)
    return wav, label

import tensorflow as tf

# tf.data.Dataset을 이용해서 데이터셋을 구성
# tf.data.Dataset.from_tensor_slices 함수에 
# return 받길 원하는 데이터를 튜플 (data, label) 형태로 넣어서 사용

# for train
train_dataset = tf.data.Dataset.from_tensor_slices((train_wav, train_label))
train_dataset = train_dataset.map(one_hot_label)
train_dataset = train_dataset.repeat().batch(batch_size=batch_size)
print(train_dataset)

# for test
test_dataset = tf.data.Dataset.from_tensor_slices((test_wav, test_label))
test_dataset = test_dataset.map(one_hot_label)
test_dataset = test_dataset.batch(batch_size=batch_size)
print(test_dataset)

## 4. 2차원 Spectrogram 데이터를 처리하는 모델 구성

   - 2차원 Spectrogram 데이터의 시간축 방향으로 Conv1D layer를 적용, 혹은 Conv2D layer를 적용 가능
   - batchnorm, dropout, dense layer 등을 이용
   - 12개의 단어 class를 구분하는 loss를 사용하고 Adam optimizer를 사용
   - 모델 가중치를 저장하는 checkpoint callback 함수 추가
   - 다양한 모델의 실험을 진행해 보시기 바랍니다.

In [ ]:
#  데이터가 2차원 audio 데이터이기 때문에 2차원 데이터를 처리하는 모델을 구성


input_tensor = layers.Input(shape=(sr, 2))

x = layers.Conv1D(32, 9, padding='same', activation='relu')(input_tensor)
x = layers.Conv1D(32, 9, padding='same', activation='relu')(x)
skip_1 = layers.MaxPool1D()(x)

x = layers.Conv1D(64, 9, padding='same', activation='relu')(skip_1)
x = layers.Conv1D(64, 9, padding='same', activation='relu')(x)
x = tf.concat([x, skip_1], -1)
skip_2 = layers.MaxPool1D()(x)

x = layers.Conv1D(128, 9, padding='same', activation='relu')(skip_2)
x = layers.Conv1D(128, 9, padding='same', activation='relu')(x)
x = layers.Conv1D(128, 9, padding='same', activation='relu')(x)
x = tf.concat([x, skip_2], -1)
skip_3 = layers.MaxPool1D()(x)

x = layers.Conv1D(256, 9, padding='same', activation='relu')(skip_3)
x = layers.Conv1D(256, 9, padding='same', activation='relu')(x)
x = layers.Conv1D(256, 9, padding='same', activation='relu')(x)
x = tf.concat([x, skip_3], -1)
x = layers.MaxPool1D()(x)
x = layers.Dropout(0.3)(x)

x = layers.Flatten()(x)
x = layers.Dense(256)(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)

output_tensor = layers.Dense(12)(x)

model_wav_skip = tf.keras.Model(input_tensor, output_tensor)

model_wav_skip.summary()

## 5. 학습 후, 학습이 어떻게 진행됐는지 그래프로 출력

   - loss, accuracy를 그래프로 표현

In [ ]:
optimizer=tf.keras.optimizers.Adam(1e-4)

model_wav.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
             optimizer=optimizer,
             metrics=['accuracy'])

In [ ]:
# the save point

checkpoint_dir = os.getenv('HOME')+'/Explolation_mj/speech_recognition/models/wav_skip'

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_dir,
                                                 save_weights_only=True,
                                                 monitor='val_loss',
                                                 mode='auto',
                                                 save_best_only=True,
                                                 verbose=1)

#30분 내외 소요 (메모리 사용량에 주의)
history_wav_skip = model_wav_skip.fit(train_dataset, epochs=max_epochs,
                    steps_per_epoch=len(train_wav) // batch_size,
                    validation_data=test_dataset,
                    validation_steps=len(test_wav) // batch_size,
                    callbacks=[cp_callback]
                    )

In [ ]:
import matplotlib.pyplot as plt

acc = history_wav.history['accuracy']
val_acc = history_wav.history['val_accuracy']

loss=history_wav.history['loss']
val_loss=history_wav.history['val_loss']

epochs_range = range(len(acc))

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()


## 6. Test dataset을 이용해서 모델의 성능을 평가

   - 저장한 weight 불러오기
   - 모델의 예측값과 정답값이 얼마나 일치하는지 확인

In [ ]:
# Evaluation 

model_wav_skip.load_weights(checkpoint_dir)
results = model_wav_skip.evaluate(test_dataset)

# loss
print("loss value: {:.3f}".format(results[0]))
# accuracy
print("accuracy value: {:.4f}%".format(results[1]*100))


In [ ]:
# Test 
# Test data 셋을 골라 직접 들어보고 모델의 예측이 맞는지 확인

inv_label_value = {v: k for k, v in label_value.items()}
batch_index = np.random.choice(len(test_wav), size=1, replace=False)

batch_xs = test_wav[batch_index]
batch_ys = test_label[batch_index]
y_pred_ = model_wav_skip(batch_xs, training=False)

print("label : ", str(inv_label_value[batch_ys[0]]))

ipd.Audio(batch_xs.reshape(8000,), rate=8000)

In [ ]:
# 위에서 확인해본 테스트셋의 라벨과 우리 모델의 실제 prediction 결과를 비교

if np.argmax(y_pred_) == batch_ys[0]:
    print("y_pred: " + str(inv_label_value[np.argmax(y_pred_)]) + '(Correct!)')
else:
    print("y_pred: " + str(inv_label_value[np.argmax(y_pred_)]) + '(Incorrect!)')

## [ 루브릭 ]

1. 음성데이터를 2차원 Spectrogram 으로 변환하여 데이터셋을 구성하였다.
	스펙트로그램 시각화 및 train/test 데이터셋 구성이 정상진행되었다.

2. 1,2차원 데이터를 처리하는 음성인식 모델이 정상 작동한다.
	스펙트로그램을 입력받은 모델이 학습과정에서 안정적으로 수렴하며, evaluation/test 단계를 무리없이 진행가능하다.

3. 테스트셋 수행결과 음성인식 모델의 Accuracy가 일정 수준에 도달하였다.
	evaluation 결과 75% 이상의 정확도를 달성하는 모델이 하나 이상 존재한다.

## [ 회 고 ]

1. 이번 프로젝트에서 어려웠던 점: 


2. 프로젝트를 진행하면서 알아낸 점 혹은 아직 모호한 점.


- 알아낸 점: 


- 모호한 점: 


3. 루브릭 평가 지표를 맞추기 위해 시도한 것들.
  - 